# Анализ решений по апелляциям на постановления ГИБДД
## п.2 Предобработка решений

Цель данного этапа:   
Предобработать решения - выделение метаданных и признаков: дата вынесения постановления, решения, город, суд, судья, инстанция, наличие представителя привлекаемого лица в суде (юрист/адвокат), номер статьи, доказательная база (наличие фото/видео материалов, свидетелей, понятых), тип решения (удовлетворить/частично/отказать...), возможно что-то ещё.

## 1. Creating df of resolusions filtrated be parameters

Существующая база решений всех судом РФ за 2010...2013 годы нахиодится в папке data/sou/ и имеет следующйю структуру: data/sou/court_name/year/resolution_n.xml

В данном проекте будут рассматриваться только дела одновременно соответствующие всем следующим критериям:
1. Вид судопроизводства: административное (Дела об админитсративных правонарушениях)
2. Этап: Апелляция
3. Статьи: главы 12 КоАП РФ (правонарушения в сфере ПДД)
4. Вид документа: решение
5. Первая инстанция: ГИБДД (инспектор ДПС)
6. Суд: все районные суды г. Москвы

In [1]:
from glob import glob
from tqdm import tqdm
import os
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import re

In [2]:
#Structure
glob('../../data/sou/*')

['../../data/sou/svobodnenskij-rajonnyj-sud-amurskaya-oblast-1759',
 '../../data/sou/motovilixinskij-rajonnyj-sud-g-permi-permskij-kraj-1277',
 '../../data/sou/arkadakskij-rajonnyj-sud-saratovskaya-oblast-190',
 '../../data/sou/oktyabrskij-rajonnyj-sud-g-tomska-tomskaya-oblast-1468',
 '../../data/sou/tunkinskij-rajonnyj-sud-respublika-buryatiya-2034',
 '../../data/sou/zarechenskij-gorodskoj-sud-penzenskaya-oblast-654',
 '../../data/sou/ulyanovskij-rajonnyj-sud-ulyanovskaya-oblast-2067',
 '../../data/sou/bogorodskij-gorodskoj-sud-nizhegorodskaya-oblast-312',
 '../../data/sou/sosnovskij-rajonnyj-sud-nizhegorodskaya-oblast-1875',
 '../../data/sou/plavskij-rajonnyj-sud-tulskaya-oblast-1584',
 '../../data/sou/sakmarskij-rajonnyj-sud-orenburgskaya-oblast-1723',
 '../../data/sou/pervomajskij-rajonnyj-sud-g-krasnodara-krasnodarskij-kraj-1547',
 '../../data/sou/staropoltavskij-rajonnyj-sud-volgogradskaya-oblast-1895',
 '../../data/sou/voskresenskij-rajonnyj-sud-nizhegorodskaya-oblast-440',
 '..

In [3]:
#Creating list of all Moscow district courts
all_courts_list = glob('../../data/sou/*') #all courts
sub = 'rajonnyj-sud-gorod-moskva' #sub name of folder of Moscow district courts
msk_districts_courts_list = sorted([s for s in all_courts_list if sub in s]) # sorted list of all Moscow district courts
print('Районных судов в Москве: {}'.format(len(msk_districts_courts_list)))
msk_districts_courts_list

Районных судов в Москве: 33


['../../data/sou/babushkinskij-rajonnyj-sud-gorod-moskva-218',
 '../../data/sou/basmannyj-rajonnyj-sud-gorod-moskva-247',
 '../../data/sou/butyrskij-rajonnyj-sud-gorod-moskva-368',
 '../../data/sou/cheremushkinskij-rajonnyj-sud-gorod-moskva-2205',
 '../../data/sou/chertanovskij-rajonnyj-sud-gorod-moskva-2225',
 '../../data/sou/dorogomilovskij-rajonnyj-sud-gorod-moskva-563',
 '../../data/sou/gagarinskij-rajonnyj-sud-gorod-moskva-462',
 '../../data/sou/golovinskij-rajonnyj-sud-gorod-moskva-479',
 '../../data/sou/izmajlovskij-rajonnyj-sud-gorod-moskva-706',
 '../../data/sou/koptevskij-rajonnyj-sud-gorod-moskva-934',
 '../../data/sou/kuncevskij-rajonnyj-sud-gorod-moskva-1047',
 '../../data/sou/kuzminskij-rajonnyj-sud-gorod-moskva-1030',
 '../../data/sou/lefortovskij-rajonnyj-sud-gorod-moskva-1148',
 '../../data/sou/lyublinskij-rajonnyj-sud-gorod-moskva-1178',
 '../../data/sou/meshhanskij-rajonnyj-sud-gorod-moskva-1232',
 '../../data/sou/nagatinskij-rajonnyj-sud-gorod-moskva-1298',
 '../../

In [4]:
%%time
#creating df of all_msk-districts_courts

cols = ['region', 'court', 'judge', 'vidpr', 'etapd', 'category', 'result', 'date', 'vid_dokumenta', 'file']
df_msk_districts = pd.DataFrame(columns=cols)


for court in msk_districts_courts_list: 
    print(court)
    path = court
    years_list = glob(os.path.join(path, '*'))
    for year in years_list:
        path = year
        files = glob(os.path.join(path, '*'))
        for file in tqdm(files):
            with open(file, 'r', encoding='utf-8') as fd:
                doc = fd.read()
            parsed_html = BeautifulSoup(doc)
            df_msk_districts = df_msk_districts.append(
                {
                    'region': parsed_html.region.text,
                    'court': parsed_html.court.text,
                    'judge': parsed_html.judge.text,
                    'vidpr': parsed_html.vidpr.text,
                    'etapd': parsed_html.etapd.text,
                    'category': parsed_html.category.text,
                    'result': parsed_html.result.text,
                    'date': parsed_html.date.text,
                    'vid_dokumenta': parsed_html.vid_dokumenta.text,
                    'file': file
                }, 
                ignore_index=True
            )

../../data/sou/babushkinskij-rajonnyj-sud-gorod-moskva-218


  0%|          | 19/4618 [00:00<00:24, 189.27it/s]

../../data/sou/basmannyj-rajonnyj-sud-gorod-moskva-247


  0%|          | 13/2707 [00:00<00:20, 128.98it/s]

../../data/sou/butyrskij-rajonnyj-sud-gorod-moskva-368


...

../../data/sou/zelenogradskij-rajonnyj-sud-gorod-moskva-668


  0%|          | 3/1178 [00:00<00:56, 20.84it/s]

../../data/sou/zyuzinskij-rajonnyj-sud-gorod-moskva-691


100%|██████████| 1266/1266 [01:01<00:00, 20.49it/s]

CPU times: user 1h 18min 6s, sys: 2min 9s, total: 1h 20min 15s
Wall time: 1h 19min 40s


In [5]:
#save to csv
df_msk_districts.to_csv('df_msk_districts.csv', index=False)

In [6]:
#load from csv back
df_msk_districts = pd.read_csv('df_msk_districts.csv')
df_msk_districts.head()

,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file
0,Москва,Бабушкинский районный суд (Город Москва),Скулина Юлия Андреевна,Гражданское,Первая инстанция,Иски о взыскании сумм по договору займа,Иск (заявление) удовлетворен (в том числе част...,30.03.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
1,Москва,Бабушкинский районный суд (Город Москва),Романцова Татьяна Алексеевна,Гражданское,Первая инстанция,Прочие исковые дела,Иск (заявление) удовлетворен (в том числе част...,09.02.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
2,Москва,Бабушкинский районный суд (Город Москва),Романцова Татьяна Алексеевна,Гражданское,Первая инстанция,"Другие споры, связанные с землепользованием",Иск (заявление) удовлетворен (в том числе част...,20.01.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
3,Москва,Бабушкинский районный суд (Город Москва),Рябова Елена Вячеславовна,Гражданское,Первая инстанция,Иски о возмещении ущерба от ДТП,Иск (заявление) удовлетворен (в том числе част...,13.07.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4,Москва,Бабушкинский районный суд (Город Москва),Мартыненко Антон Анатольевич,Уголовное,Первая инстанция,162 ч.2,Обвинительный приговор,18.05.2012,приговор,../../data/sou/babushkinskij-rajonnyj-sud-goro...


In [7]:
# create sub df of court resolutions on appeal by administrative cases
vidpr = 'Административное'
etapd = 'Апелляция'
vid_dokumenta = 'решение'
category = '12.'

df_12koap=df_msk_districts[(df_msk_districts.vidpr==vidpr)&(df_msk_districts.etapd==etapd)& \
                 (df_msk_districts.vid_dokumenta==vid_dokumenta)&
                 (df_msk_districts['category'].apply(lambda x: True if str(x).find('12.')>=0 else False) == True)]

df_12koap.head()

,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file
3125,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.27 ч.2,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
3291,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4446,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.26 ч.1,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4521,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.26 ч.1,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4728,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...


## 2. Preprocessing

### 2.1 Reveiw some statistics

In [8]:
# get statictics (absolute numbers)
def df_statistics(df):
    for col in df.columns:
        print(col)
        print('Absolute values: \n')
        print(df[col].value_counts())
        print('\n')
        print('Percentages: \n')
        print(df[col].value_counts()/len(df)*100)
        print('-'*40)

In [9]:
df_statistics(df_12koap)

region
Absolute values: 

Москва    3317
Name: region, dtype: int64


Percentages: 

Москва    100.0
Name: region, dtype: float64
----------------------------------------
court
Absolute values: 

Черемушкинский районный суд (Город Москва)    544
Зеленоградский районный суд (Город Москва)    458
Кузьминский районный суд (Город Москва)       414
Симоновский районный суд (Город Москва)       248
Хорошевский районный суд (Город Москва)       218
Бутырский районный суд (Город Москва)         190
Басманный районный суд (Город Москва)         181
Солнцевский районный суд (Город Москва)       133
Тимирязевский районный суд (Город Москва)     132
Лефортовский районный суд (Город Москва)      123
Перовский районный суд (Город Москва)         118
Останкинский районный суд (Город Москва)      104
Чертановский районный суд (Город Москва)       83
Коптевский районный суд (Город Москва)         77
Измайловский районный суд (Город Москва)       60
Головинский районный суд (Город Москва)        47
Нику

### 2.2 Editing categories' values

Видим, что в качестве значения признака category есть такие значения как '12.26 ч.1; 12.26' или '12.26; 12.26 ч.1', которые являются другим видом записи значения признака '12.26 ч.1'.
Приведём все такие записи значений в единый вид.

In [10]:
# edit categories like '12.xx ч.y; 12.xx or 12.xx; 12.xx ч.y' into '12.xx ч.y'
def edit_category(category):
    import re
    if type(category) != str: stat_part = category
    elif len(re.findall(r'12\.\d+(?:\.\d)? ч.\d+; 12\.\d+(?:\.\d)?', category)) == 1:
        stat_part = re.findall(r'(12\.\d+(?:\.\d)? ч.\d+); 12\.\d+(?:\.\d)?', category)[0]
    elif len(re.findall(r'12\.\d+(?:\.\d)?; 12\.\d+ ч.\d+(?:\.\d)?', category)) == 1:
        stat_part = re.findall(r'12\.\d+(?:\.\d)?; (12\.\d+ ч.\d+(?:\.\d)?)', category)[0]
    else: stat_part = category
    return stat_part

In [11]:
df_12koap.loc[:,'category'] = df_12koap['category'].apply(edit_category)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [12]:
df_statistics(df_12koap)

region
Absolute values: 

Москва    3317
Name: region, dtype: int64


Percentages: 

Москва    100.0
Name: region, dtype: float64
----------------------------------------
court
Absolute values: 

Черемушкинский районный суд (Город Москва)    544
Зеленоградский районный суд (Город Москва)    458
Кузьминский районный суд (Город Москва)       414
Симоновский районный суд (Город Москва)       248
Хорошевский районный суд (Город Москва)       218
Бутырский районный суд (Город Москва)         190
Басманный районный суд (Город Москва)         181
Солнцевский районный суд (Город Москва)       133
Тимирязевский районный суд (Город Москва)     132
Лефортовский районный суд (Город Москва)      123
Перовский районный суд (Город Москва)         118
Останкинский районный суд (Город Москва)      104
Чертановский районный суд (Город Москва)       83
Коптевский районный суд (Город Москва)         77
Измайловский районный суд (Город Москва)       60
Головинский районный суд (Город Москва)        47
Нику

Теперь все значения приобразованы к единому виду. Такие значения как '12.26 ч.1; 12.26' или '12.26; 12.26 ч.1' исчезли.

### 2.3 Editing results' values

Значение признака result:  
'Отменено c возвращением на новое рассмотрение' то же самое что и 'Отменено возвращением на новое рассмотрение',  
'Производство по делу прекращено' то же самое что и 'Отменено с прекращением производства'.  
Изменим все одинаковые по сути значения на те, которые используются чаще.

In [13]:
# edit the same decisions into single type
df_12koap.replace('Отменено с возвращением на новое рассмотрение', \
                  'Отменено возвращением на новое рассмотрение', inplace=True)
df_12koap.replace('Производство по делу прекращено', \
                  'Отменено с прекращением производства', inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [14]:
df_statistics(df_12koap)

region
Absolute values: 

Москва    3317
Name: region, dtype: int64


Percentages: 

Москва    100.0
Name: region, dtype: float64
----------------------------------------
court
Absolute values: 

Черемушкинский районный суд (Город Москва)    544
Зеленоградский районный суд (Город Москва)    458
Кузьминский районный суд (Город Москва)       414
Симоновский районный суд (Город Москва)       248
Хорошевский районный суд (Город Москва)       218
Бутырский районный суд (Город Москва)         190
Басманный районный суд (Город Москва)         181
Солнцевский районный суд (Город Москва)       133
Тимирязевский районный суд (Город Москва)     132
Лефортовский районный суд (Город Москва)      123
Перовский районный суд (Город Москва)         118
Останкинский районный суд (Город Москва)      104
Чертановский районный суд (Город Москва)       83
Коптевский районный суд (Город Москва)         77
Измайловский районный суд (Город Москва)       60
Головинский районный суд (Город Москва)        47
Нику

### 2.4 Add feature 'first instance'

Не все постановления по статьям 12 гл. КоАП выносятся ГИБДД. Согласно ч.1 ст. 23.1 КоАП РФ есть перечень статей, по которым постановление выносит мировой судья судебного участка, однако согласно ч. 2 той же статьи есть другой перечень статей, по которым постановление может вынести как ГИБДД, так и судья в случае, если сотруднк ГИБДД передаст это дело на рассмотрение в суд. Поэтому одназначно разделить подсудность между ГИБДД и судьями только на основании статьи невозможно. Кроме того, в пследние годы постановления по 12. гл. КоАП также стали выносить сотрудники таких организаций как МАДИ и АМПП (в Москве), в других регионах возможно тоже есть аналогичные службы, которые наряду с ГИБДД и судьями наделены полномочиями выносить постановления по 12 гл. КоАП РФ.   

Для определения первой инстанции: 
- 1) сначла проверяем докумнет по номеру статьи, если по данной статье решения однозначно выносятся только мировым судьёй судебного участка - то присваиваем этому документу в качестве 1-й инстанции - "Мировой суд", 
- 2) если же статья не входит в список статей, которые рассматриваются исключительно судьями, то 1-я инстанция определяется с помощью поиска ключевых слов и словосочетаний в тексте решений.

#### 2.4.1 Creating list of stats for mirovoy_sud only

В данной части сформируем список статей по которым постановление принимает не инспектор ГИБДД, а мировой судья.
Данный перечень содержится в тексте главы 23 КоАП РФ. Для формирования списка статей в виде, используемом в судебных актах воспользуемся регулярными выражениями.

In [15]:
# convert text with comma separaterd stats to list of standard veiw stats
def text_to_stats(text, gl='12'):
    #separate text into lines like 'parts...of stat...'
    import re
    mir_sud_cats = re.split(r'(?<='+gl+r'.\d), |(?<='+gl+r'.\d\d), |(?<='+gl+r'.\d\d.\d), |(?<=[)]), ', text)
    
    #convert stats to standard form
    stat_parts = []
    for category in mir_sud_cats:
        if len(re.findall(r' (\d(?:\.\d)?)\b', category)) == 1:
            stat = re.findall(r''+gl+r'\.\d+(?:\.\d)?', category)[0]
            part = re.findall(r' (\d(?:\.\d)?)\b', category)[0]
            stat_part = stat+' ч.'+part
            stat_parts.append(stat_part)
        if len(re.findall(r' (\d(?:\.\d)?)\b', category)) == 0:
            stat = re.findall(r''+gl+r'\.\d+(?:\.\d)?', category)[0]
            stat_part = stat
            stat_parts.append(stat_part)
        if len(re.findall(r' (\d(?:\.\d)?)\b', category)) > 1:
            stat = re.findall(r''+gl+r'\.\d+(?:\.\d)?', category)[0]
            parts = list(re.findall(r' (\d(?:\.\d)?)\b', category))
            if len(re.findall(r'\d - \d', category)) >=1:
                start = int(re.findall(r'\d - \d', category)[0][0])
                end = int(re.findall(r'\d - \d', category)[0][-1])
                parts.extend(str(i) for i in range(start, end+1))
                parts = sorted(set(parts))
            for part in parts:
                stat_part = stat+' ч.'+part
                stat_parts.append(stat_part)
    return stat_parts

Подсудность дел об административных правонарушениях

Мировой судья рассматривает дела об административных правонарушениях предусмотренные КоАП РФ в пределах компетенции установленной главой 23 КоАП РФ.

ч.1 ст. 23.1 КоАП РФ: (только статьи по Главе 12 КоАП РФ - нарущения в сфере ПДД)

... частью 4 статьи 12.2, частью 2.1 статьи 12.3, частями 1, 2, частью 3 (в случаях незаконного нанесения цветографической схемы легкового такси) статьи 12.4, частями 3, 4 - 7 статьи 12.5, частью 2 статьи 12.7, статьей 12.8, частью 7 статьи 12.9 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи), частью 3 статьи 12.10, частью 5 статьи 12.15 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), частью 3.1 статьи 12.16 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), статьями 12.24, 12.26, частями 2 и 3 статьи 12.27, статьями 12.34, 12.35, ...

In [16]:
mir_sud_text = 'частью 4 статьи 12.2, частью 2.1 статьи 12.3, частями 1, 2, частью 3 (в случаях незаконного нанесения цветографической схемы легкового такси) статьи 12.4, частями 3, 4 - 7 статьи 12.5, частью 2 статьи 12.7, статьей 12.8, частью 7 статьи 12.9 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи), частью 3 статьи 12.10, частью 5 статьи 12.15 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), частью 3.1 статьи 12.16 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), статьями 12.24, 12.26, частями 2 и 3 статьи 12.27, статьями 12.34, 12.35'

Постановления по данным частям могут выноситься сотрудниками ГИБДД, поэтому их удаляем:
<br>
частью 3 статьи 12.4 - ГИБДД
<br>
частью 7 статьи 12.9 (повторное превышение скорости) - ЦАФАП ГИБДД
<br>
частью 5 статьи 12.15 (повторный выезд на стречную полосу) - ГИБДД ЦАФАП
<br>
частью 3.1 статьи 12.16 (повторное движение во встречном направлении по односторонней дороге) - ГИБДД ЦАФАП

In [17]:
#creating list of gibdd_stats
text_gibdd_stats = 'частью 3 статьи 12.4, частью 7 статьи 12.9, частью 5 статьи 12.15, частью 3.1 статьи 12.16'
gibdd_stats = text_to_stats(text_gibdd_stats)
gibdd_stats

['12.4 ч.3', '12.9 ч.7', '12.15 ч.5', '12.16 ч.3.1']

In [18]:
#creating list of mir_sud_stats withot gibdd_stats
mir_sud_stat_parts = []
mir_sud_stat_parts = text_to_stats(mir_sud_text)
#del_gibdd_stats
mir_sud_stat_parts = set(mir_sud_stat_parts)
mir_sud_stat_parts.difference_update(set(gibdd_stats))
mir_sud_stat_parts

{'12.10 ч.3',
 '12.2 ч.4',
 '12.24',
 '12.26',
 '12.27 ч.2',
 '12.27 ч.3',
 '12.3 ч.2.1',
 '12.34',
 '12.35',
 '12.4 ч.1',
 '12.4 ч.2',
 '12.5 ч.3',
 '12.5 ч.4',
 '12.5 ч.5',
 '12.5 ч.6',
 '12.5 ч.7',
 '12.7 ч.2',
 '12.8'}

#### 2.4.2 Creating functions for text analisys

In [19]:
#check if resolutions of mirovoy sud
def is_mir_sud(text):
    import re
    if len(re.findall(r'[Сс]удебн\w+ [Уу]част\w+ |[Мм]иров\w+ [Сс]удь\w+', text)) > 0: return True
    else: return False

In [20]:
#check if resolutions of GIBDD
def is_gibdd(text):
    import re
    if len(re.findall(r'ГИБДД|ДПС|МВД|[Аа]втоматическ\w+', text)) > 0: return True
    elif (len(re.findall(r'[Ии]нспектор', text))>0)&(len(re.findall(r'12.', text))>0): return True
    else: return False

#### 2.4.3 Creating first_instance feature

In [21]:
def first_instance(file, mir_sud_stats = mir_sud_stat_parts):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    if BeautifulSoup(doc).category.text in mir_sud_stats: return 'Мировой суд'
    else:  
        text = BeautifulSoup(doc).body.text
        if is_mir_sud(text): return 'Мировой суд'
        elif is_gibdd(text): return 'ГИБДД'
        else: return 'Не определено'

In [22]:
df_12koap.loc[:,'first_instance'] = df_12koap['file'].apply(first_instance)
df_12koap[df_12koap['first_instance'] == 'Не определено']

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file,first_instance
29495,Москва,Чертановский районный суд (Город Москва),Маркина Надежда Николаевна,Административное,Апелляция,12.8 ч.1,Оставлено без изменения,29.11.2010,решение,../../data/sou/chertanovskij-rajonnyj-sud-goro...,Не определено
30700,Москва,Чертановский районный суд (Город Москва),Тюркина Галина Михайловна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,25.03.2011,решение,../../data/sou/chertanovskij-rajonnyj-sud-goro...,Не определено
66156,Москва,Кузьминский районный суд (Город Москва),Езерская Жанна Анатольевна,Административное,Апелляция,12.13 ч.2,Оставлено без изменения,17.09.2012,решение,../../data/sou/kuzminskij-rajonnyj-sud-gorod-m...,Не определено
113155,Москва,Останкинский районный суд (Город Москва),Ложкомоев Александр Юрьевич,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,08.11.2011,решение,../../data/sou/ostankinskij-rajonnyj-sud-gorod...,Не определено


Посмотрим кто выносил Постановления из тех, которые выносили не мировые судьи

In [23]:
def get_files_list(df):
    print('Всего документов: {}'.format(len(df)))
    return df.file.to_list()

In [24]:
#get text of sud act
def get_sud_text(files_list, doc_number):
    file = files_list[doc_number]
    with open(file, 'r', encoding='utf-8') as fd:
        doc = fd.read()
    parsed_html = BeautifulSoup(doc)
    return parsed_html.body.text

In [25]:
df = df_12koap[df_12koap['first_instance'] == 'Не определено']
no_mir_suf_list = get_files_list(df)

Всего документов: 4


В оставшихся 4 документах, где не определена 1-я инстанция: либо в метаданных неверно указана статья КоАП и в действительности не относится к гл. 12 либо из текста не очевидна первая инстанция

In [26]:
doc_number = 0

display(HTML(get_sud_text(no_mir_suf_list, doc_number)))

### 2.5 Creating sub df with only GIBDD first instance

In [27]:
df_gibdd = df_12koap[df_12koap.first_instance == 'ГИБДД']
print('Всего документов: {}'.format(len(df_gibdd)))
df_gibdd.head()

Всего документов: 322


,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file,first_instance
6114,Москва,Басманный районный суд (Город Москва),Карпов Артур Геннадьевич,Административное,Апелляция,12.16 ч.1,Оставлено без изменения,06.11.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
7447,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,05.07.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
8919,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,05.07.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
9005,Москва,Басманный районный суд (Город Москва),Дударь Наталия Николаевна,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,13.09.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
9784,Москва,Басманный районный суд (Город Москва),Скуридина Ирина Анатольевна,Административное,Апелляция,12.16 ч.5,Оставлено без изменения,07.03.2013,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД


### 2.6 Fill NaN

In [28]:
df_gibdd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 6114 to 172816
Data columns (total 11 columns):
region            322 non-null object
court             322 non-null object
judge             320 non-null object
vidpr             322 non-null object
etapd             322 non-null object
category          322 non-null object
result            305 non-null object
date              322 non-null object
vid_dokumenta     322 non-null object
file              322 non-null object
first_instance    322 non-null object
dtypes: object(11)
memory usage: 30.2+ KB


Есть пропуски по судьям и пропуски по результату. Судьи нас пока не сильно волнуют, а вот результат важен. Посмотрим на данные тексты и решим как можно заполнить пропуски по результатам

In [29]:
def get_result(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    if len(re.findall(r'[Пп]рекрат\w+', text)) > 0: return 'Отменено с прекращением производства'
    elif (len(re.findall(r'[Оо]тказать', text)) > 0)|\
         (len(re.findall(r'[Оо]ставить без изменени\w+', text)) > 0) : return 'Оставлено без изменения'
    elif len(re.findall(r'[Оо]тменить,*(?: \w+\W*)* дело .*ново\w* рассмотрени\w*', text)) > 0: return 'Отменено возвращением на новое рассмотрение'
    else: return 'Не определено'

In [30]:
df_gibdd.loc[(df_gibdd['result'].isna()==True)|(df_gibdd['result']== 'Не определено'), 'result'] = \
    df_gibdd.loc[(df_gibdd['result'].isna()==True), 'file'].apply(get_result)
df_gibdd[df_gibdd['result'] == 'Не определено']

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file,first_instance


In [31]:
df = df_gibdd[df_gibdd['result'] == 'Не определено']
na_list = get_files_list(df)

doc_number = 0
#display(HTML(get_sud_text(na_list, doc_number)))

Всего документов: 0


## 3. Adding keywords from text to features

### 3.1 Searching and adding 'Представитель/Адвокат/Защитник'

ПРИМЕРЫ строк для формирования правил и ислючений на основе регулярных выражений в функции проверки наличия представителя/защитника/адвоката привлекаемого лица

В текстах судебных актов упоминается представтель, даже если по факту представителя привлекаемого лица нет, проверим акты в ручную и посмотрим в каких случаях представитель имеется, а в каких случаях нет

#1. Наличие представителя:   
- Представитель Шутовой Е.В. по доверенности
- В судебном заседании представитель Горбушин жалобу поддержал.
- с участием ФИО5, представителя ФИО2,
- ФИО5 и его представитель ФИО6 доводы жалобы поддержали
- с участием заявителя ФИО3 и его представителя ФИО7
- Заявитель ФИО3 и его представитель по доверенности ФИО7
- ФИО1 и ее представитель в суд явились, доводы жалобы поддержали
- ФИО4 и его представитель в суд не явились, дело просили рассмотреть в их отсутствие (л.д.64-65).
- Представитель Болдырь Н.И. просил отменить постановление
- Представитель заявителя в судебное заседание не явился, извещен.
- Представитель ЗАО « Мерседес Бенц Рус» просил отменить постановление
- Представитель заявителя в судебное заседание не явился, просил рассмотреть дело в свое отсутствие.
- Представитель ООО « Экстенал Менеджмент Груп» просил отменить постановление
- Заявитель и его представитель по доверенности ФИО3 в судебное заседание явились

#2. Отсутствие представителя:
- помощью защитника или представителя Героев И.Б. не воспользовался
- Заслушиваются объяснения физического лица или законного представителя юридического лица
- представитель МГУП «Мосводоканал»
- законным представителем физического лица, в том числе, несовершеннолетнего, законным представителем юридического лица, защитником и представителем.
- Представитель ЦАФАП РДД ГИБДД ГУ МВД России
- Представитель ОБ ГИБДД ... по ... ... в судебное заседание не явился, извещен надлежащим образом.
- Представитель ОБ ГИБДД ... по ЗАО ... в судебное заседание явился
- Представитель ЦАФАП ОДД ГИБДД ГУ МВД России
- законным представителем юридического лица, защитником и представителем.

In [32]:
# Проверка наличия представителя/защитника/адвоката привлекаемого лица
def get_representative_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    #представитель
    if (len(re.findall(r'[Пп]редставител\w+', text)) > 0)&(len(re.findall(r'по доверенности', text)) > 0): return True
    elif len(re.findall(r'\b(?:его|е[её]) представител\w+', text)) > 0: return True
    elif len(re.findall(r'представитель \w+ жалобу', text)) > 0: return True
    elif len(re.findall(r'представител\w+ ФИО', text)) > 0: return True
    elif (len(re.findall(r'[Пп]редставител\w+ (?:\w+)+ просил отменить постановление', text)) > 0) : return True
    #&(len(re.findall(r'просил отменить постановление', text)) > 0): return True
    elif len(re.findall(r'[Пп]редставитель заявител\w+', text)) > 0: return True
    #адвокат
    elif len(re.findall(r'(?<!отсутствии) [Аа]двокат\w+', text)) > 0: return True
    #защитник
    elif (len(re.findall(r'[Зз]ащитник\w+', text)) > 0)&\
        (len(re.findall(r'[Зз]ащитник\w+ (?:и|или) [Пп]редставител\w+', text)) == 0)&\
        (len(re.findall(r'[Пп]омощ\w+ [Зз]ащитник\w+', text)) == 0): return True
    else: return False

In [33]:
df_gibdd.loc[:,'representative'] = df_gibdd['file'].apply(get_representative_existence)
df_gibdd.representative.value_counts()

False    248
True      74
Name: representative, dtype: int64

### 3.2 Searching and adding 'Фото'

In [34]:
def get_photo_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    if len(re.findall(r'[Оо]тсутстви\w+ фото\w*', text)) > 0: return 'Нет'
    elif len(re.findall(r'не представлен\w* *\w* доказательств\w* нарушен\w+(?: \w+)+, а именно,* \w+ фото\w* и видео\w*', text)) > 0: return 'Нет'
    elif len(re.findall(r'[Фф]ото- и киносъемк\w*', text)) > 0: return 'ЦАФАП'
    elif len(re.findall(r'ЦАФАП', text)) > 0: return 'ЦАФАП'
    elif len(re.findall(r'[Кк]омплекс\w* фото-видеофиксац\w+', text)) > 0: return 'ЦОДД'
    elif len(re.findall(r'[Фф]ото\w*', text)) > 0: return 'фото'
    else: return 'Нет'

In [35]:
df_gibdd.loc[:,'photo'] = df_gibdd['file'].apply(get_photo_existence)
df_gibdd.photo.value_counts()

Нет      209
фото      75
ЦАФАП     37
ЦОДД       1
Name: photo, dtype: int64

### 3.3 Searching and adding 'Видео'

In [36]:
def get_video_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    if len(re.findall(r'[Вв]идеозапис\w+ \w* не велась', text)) > 0: return 'Нет'
    elif len(re.findall(r'видеозапис\w+ не имеет\w+', text)) > 0: return 'Нет'
    elif len(re.findall(r'для запроса видеозапис\w+, поскольку', text)) > 0: return 'Нет'
    elif len(re.findall(r'не представлен\w* *\w* доказательств\w* нарушен\w+(?: \w+)+, а именно,* \w+ фото\w* и видео\w*', text)) > 0: return 'Нет'
    elif len(re.findall(r'видео\w*(?: \w*)+ не сохранил\w+', text)) > 0: return 'Нет'
    elif len(re.findall(r'[Фф]ото- и киносъемк\w*', text)) > 0: return 'ЦАФАП'
    elif len(re.findall(r'камер\w* видео\w*, установлен\w*', text)) > 0: return 'ЦАФАП'
    elif len(re.findall(r'[Фф]ото-видеофиксац\w+ в автоматическ\w+ режим\w+', text)) > 0: return 'ЦАФАП'
    elif len(re.findall(r'ЦАФАП', text)) > 0: return 'ЦАФАП'
    elif len(re.findall(r'[Кк]омплекс\w* фото-видеофиксац\w+', text)) > 0: return 'ЦОДД'
    elif len(re.findall(r'[Вв]идео\w*', text)) > 0: return 'видео'
    else: return 'Нет'

In [37]:
df_gibdd.loc[:,'video'] = df_gibdd['file'].apply(get_video_existence)
df_gibdd.video.value_counts()

Нет      250
ЦАФАП     40
видео     31
ЦОДД       1
Name: video, dtype: int64

In [38]:
#Отсутствие видеофиксации
#инспектором ДПС не представлены доказательства нарушения правил дорожного движения, а именно, данные фото и видео фиксации, не являются основаниями для отмены обжалуемого постановления.

### 3.4 Searching and adding 'ДТП'

In [39]:
def get_DTP_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    if len(re.findall(r'ДТП|[Дд]орожн\w+ транспортн\w+ прои\w+', text)) > 0: return True 
    else: return False

In [40]:
df_gibdd.loc[:,'DTP'] = df_gibdd['file'].apply(get_DTP_existence)
df_gibdd.DTP.value_counts()

False    175
True     147
Name: DTP, dtype: int64

Варианты признаков:   
Регистратор:
#была просмотрена видеозапись с регистратора

### 3.5 Searching and adding 'регистратор'

In [41]:
def get_registrator_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    if len(re.findall(r'\w*регистратор\w*', text)) > 0: return True 
    else: return False

In [42]:
df_gibdd.loc[:,'registrator'] = df_gibdd['file'].apply(get_registrator_existence)
df_gibdd.registrator.value_counts()

False    313
True       9
Name: registrator, dtype: int64

### 3.6 Searching and adding 'свидетель'

In [43]:
#original version
def get_svidetel_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    text = re.sub(r'<SPAN class="\w+">|</SPAN>', '', text) #del tags <SPAN></SPAN> from text
    if len(re.findall(r'свидетел\w{,3}\b|очевид(?!н)\w{2,4}', text)) == 0: return False    
    else:
        if len(re.findall(r'свидетел\w{,3}\b|очевид(?!н)\w{2,4}', text)) > 3: return True
        else:
            if (len(re.findall(r'(свидетел\w{,3}\b|очевид(?!н)\w{2,4})(?:,* \w+){0,6} \bне\b', text)) > 0)|\
            (len(re.findall(r'возможн\w+ (свидетел\w{,3}\b|очевид(?!н)\w{2,4})', text)) > 0)|\
            (len(re.findall(r'следовало(?: \w+,*)* \w*опросить?(?: \w+,*)* (свидетел\w{,3}\b|очевид(?!н)\w{2,4})', text)) > 0)|\
            (len(re.findall(r'без \w*опрос\w* (свидетел\w{,3}\b|очевид(?!н)\w{2,4})', text)) > 0)|\
            (len(re.findall(r'\bне\b(?: \w+){1,5} свидет\w+', text)) > 0)|\
            (len(re.findall(r'показаниями потерпевшего, свидетелей, заключениями эксперта', text)) > 0)|\
            (len(re.findall(r'не( был)?( вызван)?( и)?( опрошен)? в качестве свидетеля', text)) > 0)|\
            (len(re.findall(r'вызов\w+(?: \w+){,5} свидетел\w+', text)) > 0)|\
            (len(re.findall(r'а именно,*(?: \w+){,10} свидет\w+', text)) > 0): return False
            else: return True

### 3.7 Searching and adding 'понятой'

In [44]:
def get_ponyatoy_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    text = re.sub(r'<SPAN class="\w+">|</SPAN>', '', text) #del tags <SPAN></SPAN> from text
    if len(re.findall(r'(?<!без )[Пп]онят[оыау]\w+', text)) > 0:
        if len(re.findall(r'без понят[оыау]\w+', text)) > 0: return False    
        elif len(re.findall(r'[Пп]онят[оыау]\w+(?:,* \w+){0,6} \bне\b', text)) > 0: return False
        elif len(re.findall(r'[Оо]тсутств\w+ понят[оыау]\w+', text)) > 0: return False
        elif len(re.findall(r'\bне\b(?: \w+){,4} понят[оыау]\w+', text)) > 0: return False
        else: return True
    else: return False
    
    
df_gibdd.loc[:,'ponyatoy'] = df_gibdd['file'].apply(get_ponyatoy_existence)
df_gibdd.ponyatoy.value_counts()

False    321
True       1
Name: ponyatoy, dtype: int64

### 3.8 Searching and adding 'пассажир'

In [45]:
def get_passazhir_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    text = re.sub(r'<SPAN class="\w+">|</SPAN>', '', text) #del tags <SPAN></SPAN> from text
    if len(re.findall(r'[Пп]ассажир\w*', text)) > 0: return True
    else: return False
    
df_gibdd.loc[:,'passazhir'] = df_gibdd['file'].apply(get_passazhir_existence)
df_gibdd.passazhir.value_counts()

False    283
True      39
Name: passazhir, dtype: int64

### 3.9 Searching and adding 'пешеход'

In [46]:
def get_peshekhod_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    text = re.sub(r'<SPAN class="\w+">|</SPAN>', '', text) #del tags <SPAN></SPAN> from text
    if len(re.findall(r'[Пп]ешеход\w*', text)) > 0: return True
    else: return False
    
df_gibdd.loc[:,'peshekhod'] = df_gibdd['file'].apply(get_peshekhod_existence)
df_gibdd.peshekhod.value_counts()

False    266
True      56
Name: peshekhod, dtype: int64

### 3.10 Searching and adding 'потерпевший'

In [47]:
def get_poterpevshiy_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    text = re.sub(r'<SPAN class="\w+">|</SPAN>', '', text) #del tags <SPAN></SPAN> from text
    if len(re.findall(r'[Пп]отерпевш\w+', text)) > 0: return True
    else: return False
    
df_gibdd.loc[:,'poterpevshiy'] = df_gibdd['file'].apply(get_poterpevshiy_existence)
df_gibdd.poterpevshiy.value_counts()

False    288
True      34
Name: poterpevshiy, dtype: int64

### 3.11 Checking and adding type of judgments (start/close/cancel)

In [48]:
def get_judgment_termination(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    text = re.sub(r'<SPAN class="\w+">|</SPAN>', '', text) #del tags <SPAN></SPAN> from text  
    if re.search(r'прекращ\w+ на основани\w+|о прекращени\w+ производств\w*', text): return True
    else: return False
    
df_gibdd.loc[:,'judgment_termination'] = df_gibdd['file'].apply(get_judgment_termination)
df_gibdd.judgment_termination.value_counts()

False    278
True      44
Name: judgment_termination, dtype: int64

In [49]:
def get_judgments_type(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    text = re.sub(r'<SPAN class="\w+">|</SPAN>', '', text) #del tags <SPAN></SPAN> from text
    list_judgments = re.findall(r'(?:административн\w+ )*правонарушен\w+,* предусмотрен\w+|'+\
                                r'о наложен\w+ административн\w+ взыскан\w+|'+\
                                r'прекращено (?:на основании|в связи)|[Оо]пределение(?: \w+)* отказе в возбуждении|'+\
                                r'о прекращени\w+ производств\w*|привлеч[её]н\w* к административной ответственности|'+\
                                r'назнач\w+(?: административн\w+)* наказан\w+|'+\
                                r'вынесен\w* постановлен\w+|'+\
                                r'\bна\b решен\w+'\
                                #r'об оставлен\w+ постановлен\w+'\
                                , text)
    if len(list_judgments) == 0: return 'Не определено'   
    else: 
        if re.search(r'(?:административн\w+ )*правонарушен\w+,* предусмотрен\w+|'+\
                     r'привлеч[её]н\w* к административной ответственности|'+\
                     r'назнач\w+(?: административн\w+)* наказан\w+|'+\
                     r'вынесен\w* постановлен\w+|о наложен\w+ административн\w+ взыскан\w+'\
                     , list_judgments[0]):
            return 'Постановление по делу об АП'
        elif re.search(r'прекращено (?:на основании|в связи)|о прекращени\w+ производств\w*'\
                     , list_judgments[0]):
            return 'Постановление о прекращении дела об АП'
        elif re.search(r'[Оо]пределение(?: \w+)* отказе в возбуждении'
                       , list_judgments[0]):
            return 'Определени об отказе в возбуждении дела об АП'
        elif re.search(r'\bна\b решен\w+'
                       , list_judgments[0]):
            return 'Решение вышестоящего должностного лица' #позднее можно разбить далее на типы решений
        else: 
            return 'Другое'
    
df_gibdd.loc[:,'judgments_type'] = df_gibdd['file'].apply(get_judgments_type)
df_gibdd.judgments_type.value_counts()

Постановление по делу об АП                      277
Решение вышестоящего должностного лица            24
Постановление о прекращении дела об АП            17
Определени об отказе в возбуждении дела об АП      4
Name: judgments_type, dtype: int64

In [50]:
df = df_gibdd[df_gibdd['judgments_type'] == 'Решение вышестоящего должностного лица']
na_list = get_files_list(df)

doc_number = 23
display(HTML(get_sud_text(na_list, doc_number)))

Всего документов: 24


In [51]:
df_gibdd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 6114 to 172816
Data columns (total 22 columns):
region                  322 non-null object
court                   322 non-null object
judge                   320 non-null object
vidpr                   322 non-null object
etapd                   322 non-null object
category                322 non-null object
result                  322 non-null object
date                    322 non-null object
vid_dokumenta           322 non-null object
file                    322 non-null object
first_instance          322 non-null object
representative          322 non-null bool
photo                   322 non-null object
video                   322 non-null object
DTP                     322 non-null bool
registrator             322 non-null bool
ponyatoy                322 non-null bool
passazhir               322 non-null bool
peshekhod               322 non-null bool
poterpevshiy            322 non-null bool
judgment_termination   

## 4. Saving df_gibdd to csv

In [52]:
#save df_gibdd to csv
df_gibdd.to_csv('df_gibdd.csv', index=False)

In [53]:
#load from csv back
df_gibdd = pd.read_csv('df_gibdd.csv')
df_gibdd.head()

,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file,...,photo,video,DTP,registrator,ponyatoy,passazhir,peshekhod,poterpevshiy,judgment_termination,judgments_type
0,Москва,Басманный районный суд (Город Москва),Карпов Артур Геннадьевич,Административное,Апелляция,12.16 ч.1,Оставлено без изменения,06.11.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,...,фото,видео,True,False,False,False,False,False,False,Постановление по делу об АП
1,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,05.07.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,...,Нет,Нет,True,False,False,False,False,False,False,Постановление по делу об АП
2,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,05.07.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,...,фото,Нет,True,False,False,True,False,False,False,Постановление по делу об АП
3,Москва,Басманный районный суд (Город Москва),Дударь Наталия Николаевна,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,13.09.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,...,фото,Нет,True,False,False,False,False,False,False,Постановление по делу об АП
4,Москва,Басманный районный суд (Город Москва),Скуридина Ирина Анатольевна,Административное,Апелляция,12.16 ч.5,Оставлено без изменения,07.03.2013,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,...,фото,Нет,False,False,False,False,False,False,False,Постановление по делу об АП


In [54]:
df_gibdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 22 columns):
region                  322 non-null object
court                   322 non-null object
judge                   320 non-null object
vidpr                   322 non-null object
etapd                   322 non-null object
category                322 non-null object
result                  322 non-null object
date                    322 non-null object
vid_dokumenta           322 non-null object
file                    322 non-null object
first_instance          322 non-null object
representative          322 non-null bool
photo                   322 non-null object
video                   322 non-null object
DTP                     322 non-null bool
registrator             322 non-null bool
ponyatoy                322 non-null bool
passazhir               322 non-null bool
peshekhod               322 non-null bool
poterpevshiy            322 non-null bool
judgment_termination    322 n